# M&A SAMYANG Report 

In [16]:
import pandas as pd
import numpy as np
from pathlib import Path
import re
from datetime import datetime


# ==========================
# 셀 1 — 파일 목록 & 비교 대상 자동 선택
# ==========================

# SY 회사 drayage 폴더
DATA_DIR = Path("data") / "SY" / "drayage"

# 폴더 내 엑셀 파일 수집
files = list(DATA_DIR.glob("*.xlsx"))


# 파일명에서 날짜(앞 8자리: MMDDYYYY) 추출 → datetime.date 변환
def extract_date(path: Path):
    """
    파일명 예: 11202025 - 2.xlsx → 11/20/2025
    """
    m = re.match(r"(\d{8})", path.name)
    if not m:
        return None
    raw = m.group(1)
    try:
        return datetime.strptime(raw, "%m%d%Y").date()
    except ValueError:
        return None


# 날짜 기준 정렬 (오류 또는 없는 날짜는 최솟값 처리)
files_sorted = sorted(
    files,
    key=lambda p: extract_date(p) or datetime.min.date()
)

# 최신 2개 선정
prev_file = files_sorted[-2]      # 이전날 리포트
curr_file = files_sorted[-1]      # 최신 리포트

prev_date = extract_date(prev_file)
curr_date = extract_date(curr_file)

today = datetime.today().date()

print(f"- Today        : {today}")
print(f"- 최신 리포트     : {curr_date}  ({curr_file.name})")
print(f"- 이전 리포트     : {prev_date}  ({prev_file.name})")

- Today        : 2025-11-23
- 최신 리포트     : 2025-11-23  (11232025 - 2.xlsx)
- 이전 리포트     : 2025-11-21  (11212025 - 2.xlsx)


In [17]:
# ==========================
# 셀 2 — 전날/오늘 리포트 로드 + 공통 컬럼 정리
# ==========================

KEY = ["Customer Reference No.", "Container No."]

DISPLAY_COLS = [
    "Customer Reference No.",
    "Pick Up Location",
    "Master B/L No.",
    "Container No.",
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "EMPTY NOTICE",
    "Container Remark",
    "DELIVERY DATE",
    "Return Date",
]

# 1) 파일 읽기
df_prev = pd.read_excel(prev_file)
df_curr = pd.read_excel(curr_file)

# 2) 컬럼 공백/개행 제거
df_prev.columns = df_prev.columns.str.strip()
df_curr.columns = df_curr.columns.str.strip()

# 3) 필요 컬럼 추가 (없는 경우 NaN 생성)
for col in DISPLAY_COLS:
    if col not in df_prev.columns:
        df_prev[col] = pd.NA
    if col not in df_curr.columns:
        df_curr[col] = pd.NA

# 4) DISPLAY_COLS 순서대로 통일
df_prev = df_prev[DISPLAY_COLS]
df_curr = df_curr[DISPLAY_COLS]

print("셀2 완료 — 리포트 로드 및 컬럼 정리 완료")


셀2 완료 — 리포트 로드 및 컬럼 정리 완료


---

## 일자별 요약 리포트

In [ ]:
# ==========================
# 셀 3 — 전날/오늘 리포트 변경 사항 요약
#  1) 신규 컨테이너 개수
#  2) 값이 변경된 컨테이너 행 개수
#  3) 주요 컬럼별 변경/신규 건수
#  4) 날짜별 변화 요약 (+ Delivery / 예외)
# ==========================

import math
from datetime import datetime
import re
from collections import Counter

# 1. 인덱스를 Customer Reference No. + Container No. 로 설정
df_prev_k = df_prev.set_index(KEY)
df_curr_k = df_curr.set_index(KEY)

prev_idx = df_prev_k.index
curr_idx = df_curr_k.index

# 2. 공통 컨테이너 / 오늘만 있는 컨테이너
common_idx    = prev_idx.intersection(curr_idx)
curr_only_idx = curr_idx.difference(prev_idx)

# 3. 공통 컨테이너 기반 비교 준비
prev_c = df_prev_k.loc[common_idx]
curr_c = df_curr_k.loc[common_idx]

shared_cols = [c for c in prev_c.columns if c in curr_c.columns]
prev_c2 = prev_c[shared_cols]
curr_c2 = curr_c[shared_cols]

# 4. 셀 단위 비교 (NaN 동일 처리)
equal_mask   = (prev_c2 == curr_c2) | (prev_c2.isna() & curr_c2.isna())
changed_mask = ~equal_mask

# 5. 행 단위 변경 판단에 쓸 주요 컬럼
important_cols_for_row = [
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "DELIVERY DATE",
    "Return Date",
    "Container Remark",   # Delivery(예정) 변경도 포함
]
important_cols_for_row = [c for c in important_cols_for_row if c in shared_cols]

important_changed_mask = changed_mask[important_cols_for_row]
changed_rows_count = int(important_changed_mask.any(axis=1).sum())

# 6. 신규 컨테이너 수
added_containers = len(curr_only_idx)

# 7. 날짜/시간 컬럼별 통계
date_cols_main = [
    "Last Free Date",
    "P/U APPT DATE",
    "DELIVERY DATE",
    "Return Date",
]
date_cols_main = [c for c in date_cols_main if c in shared_cols]

column_stats = []
for col in date_cols_main + ["P/U APPT TIME"]:
    if col not in shared_cols:
        continue
    prev_col = prev_c2[col]
    curr_col = curr_c2[col]

    added_values = int((prev_col.isna() & curr_col.notna()).sum())
    changed_cells = int(changed_mask[col].sum())

    column_stats.append({
        "Column": col,
        "Added_values": added_values,
        "Changed_cells": changed_cells,
    })

# 8. Delivery(예정) 컬럼 통계 (Container Remark 기준)
if "Container Remark" in shared_cols:
    prev_rem = prev_c2["Container Remark"]
    curr_rem = curr_c2["Container Remark"]

    delivery_added = int((prev_rem.isna() & curr_rem.notna()).sum())
    delivery_changed = int(changed_mask["Container Remark"].sum())
else:
    prev_rem = curr_rem = None
    delivery_added = 0
    delivery_changed = 0

# 9. 한글 컬럼명 매핑
col_kor = {
    "Last Free Date":   "LFD",
    "P/U APPT DATE":    "픽업 날짜",
    "P/U APPT TIME":    "픽업 시간",
    "DELIVERY DATE":    "배송 완료일(Delivered)",
    "Return Date":      "리턴 날짜(Return)",
}

# 10. 상단 요약 출력
print("=== SY Drayage 리포트 변경 요약 ===")
print(f"- 신규 컨테이너 추가: {added_containers} 건")
print(f"- 값이 변경된 컨테이너 행: {changed_rows_count} 건")

for item in column_stats:
    col    = item["Column"]
    added  = item["Added_values"]
    changed = item["Changed_cells"]
    name   = col_kor.get(col, col)
    print(f"- {name}: 변경 {changed} 건 (New: {added} 건)")

print(f"- 배송 예정일(Delivery): 변경 {delivery_changed} 건 (New: {delivery_added} 건)")

print("\n--- 날짜별 요약 ---")

# ============================================
# 11. LFD / 픽업 날짜 — 날짜별 변경
# ============================================

for col in ["Last Free Date", "P/U APPT DATE"]:
    if col not in date_cols_main:
        continue

    prev_col = prev_c2[col].dropna()
    curr_col = curr_c2[col].dropna()

    if hasattr(prev_col, "dt"):
        prev_dates = prev_col.dt.date
    else:
        continue

    if hasattr(curr_col, "dt"):
        curr_dates = curr_col.dt.date
    else:
        continue

    prev_counts = prev_dates.value_counts()
    curr_counts = curr_dates.value_counts()

    all_dates = sorted(set(prev_counts.index).union(curr_counts.index))

    total_changed = 0
    lines = []

    for d in all_dates:
        prev_n = int(prev_counts.get(d, 0))
        curr_n = int(curr_counts.get(d, 0))
        if prev_n == curr_n:
            continue
        diff = curr_n - prev_n
        total_changed += abs(diff)
        sign = "+" if diff > 0 else ""
        lines.append(f"  - {d}: {prev_n} 건 → {curr_n} 건 ({sign}{diff} 건)")

    name = col_kor.get(col, col)
    print(f"[{name}] 총 {total_changed} 건 변경")
    for line in lines:
        print(line)

# ============================================
# 12. Delivery(배송 예정일) — DEL / DEL DIRECT 날짜 요약 + 예외
# ============================================

def parse_delivery_cell(val, year):
    """
    Container Remark 한 칸을 보고
    - 단일 날짜 DEL / DEL DIRECT  → (date, tag, None)
    - 복수 날짜 / 기타 텍스트     → (None, None, text)
    - MT 로 시작하는 텍스트       → (None, None, None)  # 완전 무시
    """
    if pd.isna(val):
        return None, None, None

    # datetime 이면 그대로 날짜로 사용
    if isinstance(val, (pd.Timestamp, datetime)):
        return val.date(), "DATE", None

    s = str(val).strip()
    if not s:
        return None, None, None

    up = s.upper()

    # MT 로 시작하는건 완전히 제외
    if up.startswith("MT"):
        return None, None, None

    # 날짜 패턴 (MM/DD)
    date_matches = re.findall(r"(\d{1,2})/(\d{1,2})", up)

    # 복수 날짜 or 'OR' 포함 → 예외 텍스트
    if len(date_matches) >= 2 or "OR" in up:
        return None, None, s

    # 단일 날짜 + DEL 계열
    if len(date_matches) == 1 and "DEL" in up:
        mm, dd = date_matches[0]
        try:
            d = datetime(year, int(mm), int(dd)).date()
        except ValueError:
            return None, None, s
        tag = "DEL-D" if "DIRECT" in up else "DEL"
        return d, tag, None

    # 그 외 텍스트 (예: SUPA DUPA URGENT) → 예외
    return None, None, s

def build_delivery_series(rem_series):
    dates = []
    tags = []
    ex_texts = []
    year = curr_date.year if 'curr_date' in globals() else datetime.today().year

    for v in rem_series:
        d, t, e = parse_delivery_cell(v, year)
        dates.append(d)
        tags.append(t)
        ex_texts.append(e)

    return (
        pd.Series(dates, index=rem_series.index),
        pd.Series(tags, index=rem_series.index),
        pd.Series(ex_texts, index=rem_series.index),
    )

if prev_rem is not None and curr_rem is not None:
    prev_dates_s, prev_tags_s, prev_ex_s = build_delivery_series(prev_rem)
    curr_dates_s, curr_tags_s, curr_ex_s = build_delivery_series(curr_rem)

    # ---- 날짜별 DEL / DEL-D 집계 ----
    all_dates = sorted(
        set(d for d in prev_dates_s.dropna().unique()) |
        set(d for d in curr_dates_s.dropna().unique())
    )

    total_delivery_date_changes = 0

    print("[배송 예정일(Delivery)] 날짜 기준 변경 요약")
    for d in all_dates:
        prev_mask = prev_dates_s == d
        curr_mask = curr_dates_s == d

        prev_total = int(prev_mask.sum())
        curr_total = int(curr_mask.sum())
        diff = curr_total - prev_total

        if prev_total == curr_total and diff == 0:
            continue

        total_delivery_date_changes += abs(diff)

        prev_del   = int((prev_mask & (prev_tags_s == "DEL")).sum())
        prev_deld  = int((prev_mask & (prev_tags_s == "DEL-D")).sum())
        curr_del   = int((curr_mask & (curr_tags_s == "DEL")).sum())
        curr_deld  = int((curr_mask & (curr_tags_s == "DEL-D")).sum())

        sign = "+" if diff > 0 else ""
        print(
            f"  - {d}: {prev_total} 건 → {curr_total} 건 ({sign}{diff} 건) "
            f"(DEL: {prev_del} → {curr_del}, DEL-D: {prev_deld} → {curr_deld})"
        )

    # ---- 예외 텍스트 요약 (복수 날짜, 기타 문구) ----
    prev_ex_counts = Counter(t for t in prev_ex_s.dropna())
    curr_ex_counts = Counter(t for t in curr_ex_s.dropna())
    all_ex_keys = sorted(set(prev_ex_counts.keys()) | set(curr_ex_counts.keys()))

    if all_ex_keys:
        total_ex_changed = 0
        for key in all_ex_keys:
            prev_n = prev_ex_counts.get(key, 0)
            curr_n = curr_ex_counts.get(key, 0)
            if prev_n != curr_n:
                total_ex_changed += abs(curr_n - prev_n)

        print(f"[배송 일정 예외] : 총 {total_ex_changed}건 변경")
        for key in all_ex_keys:
            prev_n = prev_ex_counts.get(key, 0)
            curr_n = curr_ex_counts.get(key, 0)
            print(f"  '{key}': 이전 {prev_n} 건, 오늘 {curr_n} 건")

# ============================================
# 13. 배송 완료일 / 리턴 날짜 — 날짜별 변화
# ============================================

for col in ["DELIVERY DATE", "Return Date"]:
    if col not in date_cols_main:
        continue

    prev_col = prev_c2[col].dropna()
    curr_col = curr_c2[col].dropna()

    if hasattr(prev_col, "dt"):
        prev_dates = prev_col.dt.date
    else:
        continue

    if hasattr(curr_col, "dt"):
        curr_dates = curr_col.dt.date
    else:
        continue

    prev_counts = prev_dates.value_counts()
    curr_counts = curr_dates.value_counts()

    all_dates = sorted(set(prev_counts.index).union(curr_counts.index))

    total_changed = 0
    lines = []

    for d in all_dates:
        prev_n = int(prev_counts.get(d, 0))
        curr_n = int(curr_counts.get(d, 0))
        if prev_n == curr_n:
            continue
        diff = curr_n - prev_n
        total_changed += abs(diff)
        sign = "+" if diff > 0 else ""
        lines.append(f"  - {d}: {prev_n} 건 → {curr_n} 건 ({sign}{diff} 건)")

    name = col_kor.get(col, col)
    print(f"[{name}] 총 {total_changed} 건 변경")
    for line in lines:
        print(line)


=== SY Drayage 리포트 변경 요약 ===
- 신규 컨테이너 추가: 0 건
- 값이 변경된 컨테이너 행: 33 건
- LFD: 변경 3 건 (New: 3 건)
- 픽업 날짜: 변경 0 건 (New: 0 건)
- 배송 완료일(Delivered): 변경 12 건 (New: 12 건)
- 리턴 날짜(Return): 변경 5 건 (New: 5 건)
- 픽업 시간: 변경 0 건 (New: 0 건)
- 배송 예정일(Delivery): 변경 25 건 (New: 1 건)

--- 날짜별 변화 요약 ---
[LFD] 총 3 건 변경
  - 2025-11-25: 4 건 → 7 건 (+3 건)
[픽업 날짜] 총 0 건 변경
[배송 예정일(Delivery)] 날짜 기준 변경 요약
  - 2025-11-22: 12 건 → 0 건 (-12 건) (DEL: 12 → 0, DEL-D: 0 → 0)
  - 2025-11-24: 8 건 → 16 건 (+8 건) (DEL: 0 → 0, DEL-D: 8 → 8)
[배송 일정 예외] : 총 8건 변경
  'DEL 11/22 OR 11/24': 이전 8 건, 오늘 0 건
  'SUPA DUPA URGENT': 이전 1 건, 오늘 1 건
[배송 완료일(Delivered)] 총 12 건 변경
  - 2025-11-22: 0 건 → 12 건 (+12 건)
[리턴 날짜(Return)] 총 5 건 변경
  - 2025-11-22: 0 건 → 5 건 (+5 건)


---

## Final Report 

In [28]:
# ==========================
# 셀 4 — 변경된 컨테이너 상세 표
#  - 기준: 전날/오늘 값이 하나라도 달라진 컨테이너
#  - Delivery = Container Remark 를 날짜/텍스트로 정리 (DEL / DEL-D / 기타 텍스트)
#  - 정렬: 1순위 LFD, 2순위 Delivery 날짜
#  - 출력 컬럼: PO#, MBL#, CNTR#, LFD, PU Date, PU Time, Delivery, Delivered, Return
#  - mask_for_style: 셀5에서 엑셀 하이라이트에 사용
# ==========================

import numpy as np
from datetime import date
import re

# 1. KEY 기준 인덱스 & 공통 구간
df_prev_k = df_prev.set_index(KEY)
df_curr_k = df_curr.set_index(KEY)

prev_idx = df_prev_k.index
curr_idx = df_curr_k.index
common_idx = prev_idx.intersection(curr_idx)

prev_c = df_prev_k.loc[common_idx]
curr_c = df_curr_k.loc[common_idx]

# 공통 컬럼만 사용
shared_cols = [c for c in prev_c.columns if c in curr_c.columns]
prev_c2 = prev_c[shared_cols]
curr_c2 = curr_c[shared_cols]

# 2. 변경 여부 (NaN 동일 취급)
equal_mask   = (prev_c2 == curr_c2) | (prev_c2.isna() & curr_c2.isna())
changed_mask = ~equal_mask

# 하나라도 값이 달라진 컨테이너만
row_changed  = changed_mask.any(axis=1)
changed_prev = prev_c2.loc[row_changed]
changed_curr = curr_c2.loc[row_changed]

print(f"변경된 컨테이너 수: {len(changed_curr)} 건\n")

# 인덱스를 풀어서 표용 DataFrame 준비
table = changed_curr.reset_index()  # KEY가 컬럼으로 들어옴 (Customer Reference No., Container No.)


# -------------------------------------------------
# 3. Container Remark → Delivery(문자열) / Delivery_sort(날짜)
#    - DEL 11/25      → 2025-11-25 (DEL)
#    - DEL DIRECT 11/25 → 2025-11-25 (DEL-D)
#    - 11/24/25 (날짜만) → 2025-11-24
#    - DEL 11/22 OR 11/24 → 예외 텍스트 그대로
#    - MT 로 시작하는 건 무시 (Delivery 집계/표시에서 제외)
# -------------------------------------------------

def parse_delivery_remark(raw, base_year):
    """
    Container Remark 를 파싱해서:
      - (단일 날짜, 타입, None)
          예: (2025-11-24, 'DEL', None)
          예: (2025-11-25, 'DEL-D', None)
      - (단일 날짜, None, None)
          예: (2025-11-24, None, None)  # 날짜만 있는 경우
      - (None, None, 예외텍스트)
          예: (None, None, 'DEL 11/22 OR 11/24')
      - (None, None, None)
          예: MT 로 시작하는 텍스트 등
    """
    if pd.isna(raw):
        return None, None, None

    s_orig = str(raw).strip()
    if not s_orig:
        return None, None, None

    s = s_orig.upper()

    # MT 로 시작하는 건 Delivery 계산/표시에서 제외
    if s.startswith("MT "):
        return None, None, None

    # DEL / DEL DIRECT 태그
    tag = None
    if "DEL DIRECT" in s:
        tag = "DEL-D"
    elif "DEL" in s:
        tag = "DEL"

    # MM/DD 또는 MM/DD/YY 패턴
    date_matches = re.findall(r"(\d{1,2})/(\d{1,2})(?:/(\d{2,4}))?", s)

    # 날짜가 2개 이상이거나 OR 포함 → 예외 텍스트로만 처리
    if len(date_matches) >= 2 or " OR " in s:
        return None, None, s_orig

    # DEL / DEL-D + 날짜 1개 → 실제 날짜로 변환
    if tag and len(date_matches) == 1:
        mm, dd, yy = date_matches[0]
        mm = int(mm)
        dd = int(dd)

        if yy:
            yy_i = int(yy)
            if yy_i < 100:
                yy_i += 2000
            year = yy_i
        else:
            year = base_year

        try:
            d = date(year, mm, dd)
            return d, tag, None
        except ValueError:
            # 이상한 날짜면 그냥 텍스트 예외 처리
            return None, None, s_orig

    # 여기까지 왔는데 날짜 패턴이 없으면,
    #  "2025-11-24 00:00:00" 같은 순수 날짜/시간 텍스트인 경우 처리
    try:
        dt = pd.to_datetime(s_orig, errors="raise")
        return dt.date(), None, None
    except Exception:
        pass

    # 그 밖의 텍스트 (예: SUPA DUPA URGENT 등) → 예외로 남김
    return None, None, s_orig


base_year = curr_date.year  # 셀1에서 최신 리포트 날짜 연도

delivery_dates = []
delivery_texts = []

for remark in table["Container Remark"]:
    d, tag, ex = parse_delivery_remark(remark, base_year)

    if d is not None:
        base_str = d.strftime("%Y-%m-%d")
        if tag == "DEL-D":
            delivery_texts.append(f"{base_str} (DEL-D)")
        elif tag == "DEL":
            delivery_texts.append(f"{base_str} (DEL)")
        else:
            # 그냥 날짜만 있는 경우
            delivery_texts.append(base_str)
        delivery_dates.append(d)
    elif ex is not None:
        # 예외 텍스트 (복수 날짜, SUPA DUPA URGENT 등) 그대로 출력
        delivery_texts.append(ex)
        delivery_dates.append(None)
    else:
        delivery_texts.append("")
        delivery_dates.append(None)


# -------------------------------------------------
# 4. 표 구성: PO, MBL, CNTR, 날짜/시간, Delivered, Return + Delivery
# -------------------------------------------------

changed_table = pd.DataFrame({
    "PO#":   table["Customer Reference No."],
    "MBL#":  table["Master B/L No."],
    "CNTR#": table["Container No."],
    "LFD":      table["Last Free Date"],
    "PU Date":  table["P/U APPT DATE"],
    "PU Time":  table["P/U APPT TIME"],
    "Delivery": delivery_texts,
    "Delivered": table["DELIVERY DATE"],
    "Return":    table["Return Date"],
})

# 날짜 컬럼들을 전부 YYYY-MM-DD 문자열로 변환 + NaT/NaN 은 빈칸
for col in ["LFD", "PU Date", "Delivered", "Return"]:
    dt = pd.to_datetime(changed_table[col], errors="coerce")
    changed_table[col] = dt.dt.strftime("%Y-%m-%d").fillna("")

# 시간 컬럼은 HH:MM, NaT → ""
pu_time_dt = pd.to_datetime(changed_table["PU Time"], errors="coerce")
changed_table["PU Time"] = pu_time_dt.dt.strftime("%H:%M").fillna("")

# Delivery 정렬용 날짜 (DEL/DEL-D 또는 순수 날짜인 경우만 사용)
delivery_sort = pd.to_datetime(pd.Series(delivery_dates), errors="coerce")
changed_table["Delivery_sort"] = delivery_sort.dt.date

# LFD 정렬용
lfd_sort = pd.to_datetime(changed_table["LFD"], errors="coerce")
changed_table["LFD_sort"] = lfd_sort.dt.date

# -------------------------------------------------
# 5. 정렬: 1순위 LFD, 2순위 Delivery_sort (NaN 은 맨 뒤)
# -------------------------------------------------
changed_table = (
    changed_table
    .sort_values(
        by=["LFD_sort", "Delivery_sort"],
        ascending=[True, True],
    )
    .drop(columns=["LFD_sort", "Delivery_sort"])
    .reset_index(drop=True)
)

# index 1부터 시작
changed_table.index = range(1, len(changed_table) + 1)

# -------------------------------------------------
# 6. 하이라이트용 마스크 (어디가 바뀐 셀인지)
#    - LFD, PU Date, PU Time, Delivery(Container Remark), Delivered, Return 만 사용
# -------------------------------------------------

mask_for_style = pd.DataFrame(
    False, index=changed_table.index, columns=changed_table.columns
)

# changed_mask 중에서 우리가 보고 싶은 컬럼만
mask_cols = [c for c in [
    "Last Free Date",
    "P/U APPT DATE",
    "P/U APPT TIME",
    "Container Remark",   # Delivery 로 표시됨
    "DELIVERY DATE",
    "Return Date",
] if c in changed_mask.columns]

mask_src = changed_mask[mask_cols].loc[row_changed]

# 원래 컬럼명 → 표시 컬럼명 매핑
col_map = {
    "Last Free Date":   "LFD",
    "P/U APPT DATE":    "PU Date",
    "P/U APPT TIME":    "PU Time",
    "Container Remark": "Delivery",
    "DELIVERY DATE":    "Delivered",
    "Return Date":      "Return",
}

for idx, row in changed_table.iterrows():
    key = (row["PO#"], row["CNTR#"])
    if key not in mask_src.index:
        continue

    src_row = mask_src.loc[key]

    for src_col, display_col in col_map.items():
        if src_col not in src_row.index:
            continue
        if display_col not in mask_for_style.columns:
            continue
        if bool(src_row[src_col]):
            mask_for_style.at[idx, display_col] = True

# -------------------------------------------------
# 7. 노트북 출력용 스타일 (깃허브에선 색 안 먹는 거 이미 알고 있음)
# -------------------------------------------------

def highlight_changes(_df):
    styles = np.where(
        mask_for_style.values,
        "background-color: #fff3b0; color: red;",
        "",
    )
    return pd.DataFrame(styles, index=_df.index, columns=_df.columns)

styled_table = (
    changed_table.style
    .set_table_styles(
        [{"selector": "th", "props": [("text-align", "center")]}]
    )
    .apply(highlight_changes, axis=None)
)

styled_table


변경된 컨테이너 수: 33 건



,PO#,MBL#,CNTR#,LFD,PU Date,PU Time,Delivery,Delivered,Return
1,25-6828,ZIMUSEL71188158,JXLU4660830,2025-11-18,2025-11-21,11:00,2025-11-24,,
2,25-6835,ZIMUSEL71188171,CAAU7789800,2025-11-18,2025-11-18,14:00,,2025-11-20,2025-11-22
3,25-6846,ZIMUSEL71188144,ZCSU7923334,2025-11-18,2025-11-18,15:00,,2025-11-20,2025-11-22
4,25-5984,ONEYSELFG9174903,ONEU5124335,2025-11-19,2025-11-19,14:00,,2025-11-21,2025-11-22
5,25-5985,ONEYSELFG9174904,TCNU6196339,2025-11-19,2025-11-19,15:00,,2025-11-21,2025-11-22
6,25-6829,ZIMUSEL71188186,JXLU6326186,2025-11-19,2025-11-21,10:00,,2025-11-22,
7,25-6833,ZIMUSEL71188178,JXLU6316716,2025-11-19,2025-11-21,10:00,,2025-11-22,
8,25-6838,ZIMUSEL71188170,CAAU6870331,2025-11-19,2025-11-21,10:00,,2025-11-22,
9,25-6845,ZIMUSEL71188172,ZCSU6770544,2025-11-19,2025-11-20,23:00,,2025-11-22,
10,25-6847,ZIMUSEL71188169,JXLU6147010,2025-11-19,2025-11-20,23:00,,2025-11-22,


---

## Save

In [30]:
# ==========================
# 셀 5 — 정렬해서 엑셀 파일로 저장
#  - 정렬: 1) LFD, 2) Delivery(DEL/DEL-D 날짜) 오름차순
#  - Delivery_sort 는 정렬용 숨김 컬럼 (엑셀에 안 보이게 저장)
# ==========================

from pathlib import Path
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
import re

# 셀4에서 사용한 표시컬럼 → 원본 컬럼 매핑
display_to_orig = {
    "LFD": "Last Free Date",
    "PU Date": "P/U APPT DATE",
    "PU Time": "P/U APPT TIME",
    "Delivery": "Container Remark",   # Remark 기반 (DEL/DEL-D 등)
    "Delivered": "DELIVERY DATE",
    "Return": "Return Date",
}

OUTPUT_DIR = Path("output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

date_str = curr_date.strftime("%Y%m%d")
output_path = OUTPUT_DIR / f"sy_drayage_diff_{date_str}.xlsx"

# 1) 저장용 DataFrame 복사
df_to_save = changed_table.copy()

# 1-1) LFD를 날짜로 캐스팅 (혹시 모를 dtype 꼬임 방지)
df_to_save["LFD"] = pd.to_datetime(df_to_save["LFD"], errors="coerce")

# 1-2) Delivery 에서 정렬용 날짜 뽑기 (문자열 안의 YYYY-MM-DD)
def extract_delivery_date(val):
    if not isinstance(val, str):
        return None
    m = re.search(r"\d{4}-\d{2}-\d{2}", val)
    if not m:
        return None
    return m.group(0)

df_to_save["Delivery_sort"] = pd.to_datetime(
    df_to_save["Delivery"].apply(extract_delivery_date),
    errors="coerce"
)

# 1-3) LFD → Delivery_sort 기준 정렬
df_to_save = df_to_save.sort_values(
    by=["LFD", "Delivery_sort"],
    ascending=[True, True],
    na_position="last",
)

# 1-4) 엑셀에는 Delivery_sort 안 보이게 컬럼 제거
export_df = df_to_save.drop(columns=["Delivery_sort"])

# 2) 엑셀로 저장
export_df.to_excel(output_path, index=True, sheet_name="SY_Drayage_Diff")

# 3) 스타일 적용
wb = load_workbook(output_path)
ws = wb.active

# 3-1) 헤더 가운데 정렬
max_col = ws.max_column
for col in range(1, max_col + 1):
    cell = ws.cell(row=1, column=col)
    cell.alignment = Alignment(horizontal="center", vertical="center")

# 3-2) 변경된 셀 하이라이트
fill_changed = PatternFill(fill_type="solid", fgColor="FFF3B0")
font_changed = Font(color="FF0000")

for excel_row_idx, (df_idx, row) in enumerate(export_df.iterrows(), start=2):
    po   = row["PO#"]
    cntr = row["CNTR#"]
    key = (po, cntr)

    # prev / curr 에 없는 키는 스킵
    if key not in df_prev_k.index or key not in df_curr_k.index:
        continue

    prev_row = df_prev_k.loc[key]
    curr_row = df_curr_k.loc[key]

    for col_offset, col_name in enumerate(export_df.columns, start=2):  # 2열부터 데이터
        if col_name not in display_to_orig:
            continue

        orig_col = display_to_orig[col_name]
        prev_val = prev_row.get(orig_col, pd.NA)
        curr_val = curr_row.get(orig_col, pd.NA)

        same = (
            (pd.isna(prev_val) and pd.isna(curr_val))
            or (prev_val == curr_val)
        )
        if not same:
            cell = ws.cell(row=excel_row_idx, column=col_offset)
            cell.fill = fill_changed
            cell.font = font_changed

# 4) 열 너비 자동 조정 (#### 안 뜨게)
for column_cells in ws.columns:
    max_length = 0
    col_letter = column_cells[0].column_letter
    for cell in column_cells:
        if cell.value is None:
            continue
        s = str(cell.value)
        if len(s) > max_length:
            max_length = len(s)
    ws.column_dimensions[col_letter].width = max_length + 2

wb.save(output_path)
print(f"정렬까지 완료해서 저장: {output_path}")


정렬까지 완료해서 저장: output/sy_drayage_diff_20251123.xlsx
